In [30]:
import datetime
import ee 
import geemap
import colorcet as cc
from data_utils.pygeoboundaries import get_adm_ee
from google.cloud import storage
from data_utils.monitor_tasks import monitor_tasks
import csv
from io import StringIO
from collections import Counter

In [31]:
ee.Authenticate()

True

In [32]:
cloud_project = 'hotspotstoplight'

In [33]:
ee.Initialize(project=cloud_project)

In [52]:
place_name = "Costa Rica"

scale = 90

snake_case_place_name = place_name.replace(' ', '_').lower()

aoi = get_adm_ee(territories=place_name, adm='ADM0')
bbox = aoi.geometry().bounds()

In [35]:
# Applies scaling factors.
def apply_scale_factors(image):
    # Scale and offset values for optical bands
    optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    
    # Scale and offset values for thermal bands
    thermal_bands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
    
    # Add scaled bands to the original image
    return image.addBands(optical_bands, None, True).addBands(thermal_bands, None, True)

# Function to Mask Clouds and Cloud Shadows in Landsat 8 Imagery
def cloud_mask(image):
    # Define cloud shadow and cloud bitmasks (Bits 3 and 5)
    cloud_shadow_bitmask = 1 << 3
    cloud_bitmask = 1 << 5
    
    # Select the Quality Assessment (QA) band for pixel quality information
    qa = image.select('QA_PIXEL')
    
    # Create a binary mask to identify clear conditions (both cloud and cloud shadow bits set to 0)
    mask = qa.bitwiseAnd(cloud_shadow_bitmask).eq(0).And(qa.bitwiseAnd(cloud_bitmask).eq(0))
    
    # Update the original image, masking out cloud and cloud shadow-affected pixels
    return image.updateMask(mask)

In [36]:
bucket_name = f'hotspotstoplight_heatmapping'
directory_name = f'data/{snake_case_place_name}/inputs/'

storage_client = storage.Client(project=cloud_project)
bucket = storage_client.bucket(bucket_name)
blob = bucket.blob(directory_name)
blob.upload_from_string('', content_type='application/x-www-form-urlencoded;charset=UTF-8')

file_prefix="ndvi_min_max"

year = 2022

gcs_bucket = bucket_name

In [37]:
def export_ndvi_min_max(year, bbox, scale, gcs_bucket, snake_case_place_name, file_prefix="ndvi_min_max"):
    try:
        startDate = ee.Date.fromYMD(year, 1, 1)
        endDate = ee.Date.fromYMD(year, 12, 31)

        # Filter the collection for the given year and bounds
        imageCollection = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2") \
            .filterBounds(bbox) \
            .filterDate(startDate, endDate) \
            .map(apply_scale_factors) \
            .map(cloud_mask)

        # Calculate NDVI for the entire collection
        ndviCollection = imageCollection.map(lambda image: image.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI'))

        # Reduce the collection to get min and max NDVI values
        ndvi_min = ndviCollection.reduce(ee.Reducer.min()).reduceRegion(reducer=ee.Reducer.min(), geometry=bbox, scale=scale, maxPixels=1e9)
        ndvi_max = ndviCollection.reduce(ee.Reducer.max()).reduceRegion(reducer=ee.Reducer.max(), geometry=bbox, scale=scale, maxPixels=1e9)

        # Create a feature to export
        feature = ee.Feature(None, {'ndvi_min': ndvi_min.get('NDVI_min'), 'ndvi_max': ndvi_max.get('NDVI_max')})

        # Create and start the export task with the specified fileNamePrefix
        task = ee.batch.Export.table.toCloudStorage(
            collection=ee.FeatureCollection([feature]),
            description=f'{file_prefix}_{year}',
            bucket=gcs_bucket,
            fileNamePrefix=f'data/{snake_case_place_name}/inputs/{file_prefix}_{year}',
            fileFormat='CSV'
        )
        task.start()

        # Print statements confirming the task has started
        print(f"Starting export task for NDVI min/max values of year {year}.")

        # Return the task object
        return task

    except Exception as e:
        print(f"An error occurred while starting the export task for year {year}: {e}")
        return None

In [54]:
def download_ndvi_data_for_year(year, cloud_project, bucket_name, snake_case_place_name):
    # Initialize the Google Cloud Storage client
    storage_client = storage.Client(project=cloud_project)
    bucket = storage_client.bucket(bucket_name)
    
    # Define the blob's name to include the full path
    blob_name = f'data/{snake_case_place_name}/inputs/ndvi_min_max_{year}.csv'
    blob = bucket.blob(blob_name)
    
    # Download the data as a string
    ndvi_data_csv = blob.download_as_string()
    
    # Parse the CSV data
    ndvi_data = csv.reader(StringIO(ndvi_data_csv.decode('utf-8')))
    rows = list(ndvi_data)
    
    # Extract NDVI min and max values
    # Assuming the first row after the header contains NDVI min and the second row contains NDVI max
    # Note: This assumes row 1 is headers
    ndvi_min = float(rows[1][1])
    ndvi_max = float(rows[1][2])
    
    return ndvi_min, ndvi_max

In [55]:
def process_year(year, ndvi_min, ndvi_max):
    # Define the start and end dates for the year
    startDate = ee.Date.fromYMD(year, 1, 1)
    endDate = ee.Date.fromYMD(year, 12, 31)

    # Import and preprocess Landsat 8 imagery for the year
    imageCollection = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2") \
                .filterBounds(bbox) \
                .filterDate(startDate, endDate) \
                .map(apply_scale_factors) \
                .map(cloud_mask)

    # Function to calculate LST for each image in the collection
    def calculate_lst(image):
        # Calculate Normalized Difference Vegetation Index (NDVI)
        ndvi = image.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')
        
        # Use the passed ndvi_min and ndvi_max directly instead of calculating them
        # Convert them to ee.Number since they are likely passed as Python primitives
        ndvi_min_ee = ee.Number(ndvi_min)
        ndvi_max_ee = ee.Number(ndvi_max)

        # Fraction of Vegetation (FV) Calculation
        fv = ee.Image().expression(
            "(ndvi - ndvi_min) / (ndvi_max - ndvi_min)",
            {
                'ndvi': ndvi,
                'ndvi_max': ndvi_max_ee,
                'ndvi_min': ndvi_min_ee
            }
        ).pow(2).rename('FV')

        # Emissivity Calculation
        em = fv.multiply(ee.Number(0.004)).add(ee.Number(0.986)).rename('EM')

        # Select Thermal Band (Band 10) and Rename It
        thermal = image.select('ST_B10').rename('thermal')

        # Land Surface Temperature (LST) Calculation
        lst = thermal.expression(
            '(TB / (1 + (0.00115 * (TB / 1.438)) * log(em))) - 273.15', {
                'TB': thermal.select('thermal'), # Select the thermal band
                'em': em # Assign emissivity
            }).rename('LST')

        return lst

    # Apply the calculate_lst function to each image in the collection
    lstCollection = imageCollection.map(calculate_lst)

    lstMean = lstCollection.mean().clip(bbox)

    # Create a binary image for each image in the collection where 1 indicates LST >= 33 and 0 otherwise
    hotDaysCollection = lstCollection.map(lambda image: image.gte(33))

    # Sum all the binary images in the collection to get the total number of hot days in the year
    hotDaysYear = hotDaysCollection.sum()

    landcover = ee.Image("ESA/WorldCover/v100/2020").select('Map').clip(bbox)

    dem = ee.ImageCollection("projects/sat-io/open-datasets/FABDEM").mosaic().clip(bbox)

    image_for_sampling = landcover.rename('landcover') \
        .addBands(dem.rename('elevation')) \
        .addBands(ee.Image.pixelLonLat()) \
        .addBands(lstMean.rename('lst_mean')) \
        .addBands(hotDaysYear.rename('hot_days')
                  ) 
        
    print("Sampling image band names", image_for_sampling.bandNames().getInfo())

    return image_for_sampling

In [58]:
years = range(2018, 2024)

def process_for_year(year, cloud_project, bucket_name, snake_case_place_name):

    ndvi_min, ndvi_max = download_ndvi_data_for_year(year, cloud_project, bucket_name, snake_case_place_name)
    image_collection = process_year(year, ndvi_min, ndvi_max)

    return image_collection


for year in years:
    export_ndvi_min_max(year, bbox, scale, gcs_bucket, snake_case_place_name)


image_list = []

for year in years:
    image = process_for_year(year, cloud_project, bucket_name, snake_case_place_name)
    image_list.append(image)

image_collections = ee.ImageCollection.fromImages(image_list)

Starting export task for NDVI min/max values of year 2018.
Starting export task for NDVI min/max values of year 2019.
Starting export task for NDVI min/max values of year 2020.
Starting export task for NDVI min/max values of year 2021.
Starting export task for NDVI min/max values of year 2022.
Starting export task for NDVI min/max values of year 2023.
Starting export task for NDVI min/max values of year 2018.
Sampling image band names ['landcover', 'elevation', 'longitude', 'latitude', 'lst_mean', 'hot_days']
Starting export task for NDVI min/max values of year 2019.
Sampling image band names ['landcover', 'elevation', 'longitude', 'latitude', 'lst_mean', 'hot_days']
Starting export task for NDVI min/max values of year 2020.
Sampling image band names ['landcover', 'elevation', 'longitude', 'latitude', 'lst_mean', 'hot_days']
Starting export task for NDVI min/max values of year 2021.
Sampling image band names ['landcover', 'elevation', 'longitude', 'latitude', 'lst_mean', 'hot_days']
St

In [70]:
# Sample the 'landcover' band of the image within the specified bounding box
sample = image_collections.first().select('landcover').sample(
  region=bbox,
  scale=10,  # Adjust scale as needed to match your image resolution and the granularity you need
  numPixels=10000,  # Number of pixels to sample for estimating class distribution
  seed=0,
  geometries=False  # Geometry information not required for this step
)

# Extract land cover class values from the sample
# Note: The band name inside aggregate_array should match your band of interest; adjust if necessary
sampled_values = sample.aggregate_array('landcover').getInfo()

# Calculate the histogram (frequency of each class)
class_histogram = Counter(sampled_values)

print("Class histogram", class_histogram)

# Total number of samples you aim to distribute across classes
total_samples = 5000

# Determine class values (unique land cover classes) and their proportional sample sizes
class_values = list(class_histogram.keys())
class_points = [int((freq / sum(class_histogram.values())) * total_samples) for freq in class_histogram.values()]

Class histogram Counter({10: 1551, 80: 1144, 30: 373, 40: 25, 95: 14, 50: 12, 60: 11, 90: 9, 20: 4})


In [71]:
class_band = 'landcover'

n_images = image_collections.size().getInfo()
samples_per_image = total_samples // n_images

# Function to apply stratified sampling to an image
def stratified_sample_per_image(image):
    # Perform stratified sampling
    stratified_sample = image.stratifiedSample(
        numPoints=samples_per_image,
        classBand=class_band,
        region=bbox,
        scale=30,
        seed=0,
        classValues=class_values,
        classPoints=class_points,
        geometries=True
    )
    # Return the sample
    return stratified_sample

# Apply the function to each image in the collection
samples = image_collections.map(stratified_sample_per_image)

# Flatten the collection of collections into a single FeatureCollection
stratified_sample = samples.flatten()

In [72]:
# Split the data into training and testing
training_sample = stratified_sample.randomColumn()
training = training_sample.filter(ee.Filter.lt('random', 0.7))
testing = training_sample.filter(ee.Filter.gte('random', 0.7))

# Train the Random Forest regression model
inputProperties=['longitude', 'latitude', 'landcover', 'elevation']
numTrees = 10  # Number of trees in the Random Forest
regressor = ee.Classifier.smileRandomForest(numTrees).setOutputMode('REGRESSION').train(
    training, 
    classProperty='hot_days', 
    inputProperties=inputProperties
)


In [78]:
# Take the first image from the filtered collection as the recent_image
recent_image = image_collections.filter(ee.Filter.calendarRange(2020, 2020, 'year')).first()

# Proceed with the classification
predicted_image = recent_image.select(inputProperties).classify(regressor)

# Calculate the difference
difference = recent_image.select('hot_days').subtract(predicted_image).rename('difference')

In [79]:
vizParams = {
    'min': 0,
    'max': 25,
    'palette': cc.fire
}

m = geemap.Map()
m.centerObject(aoi, 8)
m.add("basemap_selector")
m.add("layer_manager")
# m.addLayer(image.select('landcover'), {}, 'Land Cover')
m.addLayer(recent_image.select('hot_days').clip(aoi), vizParams, 'Actual Max LST')
m.addLayer(predicted_image.clip(aoi), vizParams, 'Predicted LST')
m.addLayer(difference, {'min': -10, 'max': 10, 'palette': cc.cwr}, 'Difference')
m

Map(center=[9.973546567083341, -84.19370076516422], controls=(WidgetControl(options=['position', 'transparent_…

In [80]:
# Assuming 'max_lst' is your actual maximum LST image and 'predicted_image' contains the predictions
# Calculate the squared difference between actual and predicted LST
squared_difference = recent_image.select('hot_days').subtract(predicted_image).pow(2).rename('difference')

# Reduce the squared differences to get the mean squared difference over your area of interest (aoi)
mean_squared_error = squared_difference.reduceRegion(
    reducer=ee.Reducer.mean(),
    geometry=bbox,
    scale=scale,  # Adjust scale to match your dataset's resolution
    maxPixels=1e14
)

# Calculate the square root of the mean squared error to get the RMSE
rmse = mean_squared_error.getInfo()['difference'] ** 0.5

print('RMSE:', rmse)

RMSE: 2.119165613663734
